# Camera Trap Processing Thesis

Author: Jameson Andrews\
EDITS:\
09/13/2023, Initial Creation, Jameson

## Load model

In [1]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import models
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19

preTrainedModel = VGG19()
preTrainedModel.trainable = False
preTrainedModel.summary()


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-11-01 21:54:44.804435: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-01 21:54:44.804583: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## Create complete Model with Custom output Layer

In [2]:
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

completeModel = models.Sequential()
completeModel.add(preTrainedModel)
completeModel.add(Dense(100, activation = "relu"))
# completeModel.add(Dense(75, activation = "relu"))
# completeModel.add(Dense(50, activation = "relu"))
# completeModel.add(Dense(25, activation = "relu"))
completeModel.add(Dense(1, activation = "sigmoid"))
completeModel.summary()
completeModel.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1000)              143667240 
                                                                 
 dense (Dense)               (None, 100)               100100    
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 143,767,441
Trainable params: 100,201
Non-trainable params: 143,667,240
_________________________________________________________________


/Users/jamesonandrews/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## Orgainze test and training data
Be sure to change file contents

In [3]:
trainingValues = [0] * 989 + [1] * 1008
testValues = [1] * 160 + [0] * 154
trainImages = keras.utils.image_dataset_from_directory( #Takes given directory and automatically organizes it based on subdirectories
    directory='trainingData',
    labels = trainingValues,
    label_mode='binary',
    batch_size=32,
    image_size=(224, 224))

# Test Images not configured
# testImages = keras.utils.image_dataset_from_directory(
#     directory='testData',
#     labels= testValues,
#     label_mode='binary',
#     batch_size=32,
#     image_size=(224, 224))
# print (trainImages)

Found 1997 files belonging to 2 classes.


## Prediction before training

In [4]:
testPicture = load_img('../deerCrop.jpeg', target_size=(224,224))
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)

2023-11-01 21:54:48.186186: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-11-01 21:54:48.275100: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 1s/step
[[0.5159187]]


## Train Last two layers

In [5]:
completeModel.fit(trainImages, epochs=10, batch_size=100)

Epoch 1/10


2023-11-01 21:54:49.879903: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - 23s 319ms/step - loss: 0.6669 - accuracy: 0.6945
Epoch 2/10
63/63 [==============================] - 20s 313ms/step - loss: 0.5766 - accuracy: 0.7506
Epoch 3/10
63/63 [==============================] - 20s 313ms/step - loss: 0.5003 - accuracy: 0.7687
Epoch 4/10
63/63 [==============================] - 20s 313ms/step - loss: 0.4566 - accuracy: 0.7872
Epoch 5/10
63/63 [==============================] - 20s 313ms/step - loss: 0.4280 - accuracy: 0.8137
Epoch 6/10
63/63 [==============================] - 20s 313ms/step - loss: 0.4061 - accuracy: 0.8252
Epoch 7/10
63/63 [==============================] - 20s 313ms/step - loss: 0.3883 - accuracy: 0.8292
Epoch 8/10
63/63 [==============================] - 20s 313ms/step - loss: 0.3733 - accuracy: 0.8453
Epoch 9/10
63/63 [==============================] - 20s 313ms/step - loss: 0.3628 - accuracy: 0.8493
Epoch 10/10
63/63 [==============================] - 20s 313ms/step - loss: 0.3502 - accuracy: 0.8553


## Test Some data

In [6]:
# score = completeModel.evaluate(testImages)
# print ("loss = ", score[0])
# print ("accuracy = ", score[1])

## See what happens with a deer image

In [7]:
from PIL import Image
testPicture = load_img('../deerCrop.jpeg', target_size=(224,224))
# testPicture = load_img('../deerNightCrop.jpg', target_size=(224,224))
# testPicture = load_img('../foxNightCrop2.jpg', target_size=(224,224))
# testPicture = load_img('../possumNightCrop.jpg', target_size=(224,224))
# testPicture = load_img('../downloadRabbit.jpeg', target_size=(224,224))
# testPicture = load_img('../rabbitNightCrop1.jpg', target_size=(224,224))
testPicture.show()
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)


# 0 is deer, 1 is fox, 2 is Possum, 3 is rabbit, 4 is squirrel

1/1 [==============================] - 0s 22ms/step
[[0.00603731]]


## Save the model for export

In [8]:
completeModel.save("coyoteModel.keras")

## Check test data for each animal

In [9]:
# import required module
import os
# assign directory
directory = 'extraTest/testDeer'
 
# iterate over files in
# that directory
# 0 is positives 1 is negativess
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1

print (results)

extraTest/testDeer/1705.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 18ms/step
[[0.16876087]]
extraTest/testDeer/1687.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.20185657]]
extraTest/testDeer/1703.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.03960832]]
extraTest/testDeer/1692.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.5106069]]
extraTest/testDeer/1673.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.0325455]]
extraTest/testDeer/1694.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.25434276]]
extraTest/testDeer/1681.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.00769555]]
extraTest/testDeer/1679.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.04258237]]
extraTest/testDeer/1698.jp

In [10]:
directory = 'extraTest/testFox'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testFox/3267.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg


1/1 [==============================] - 0s 13ms/step
[[0.5360255]]
extraTest/testFox/3282.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.29279253]]
extraTest/testFox/3295.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.05983261]]
extraTest/testFox/3257.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.41600767]]
extraTest/testFox/3254.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.87070113]]
extraTest/testFox/3278.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.03084727]]
extraTest/testFox/3270.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.14367759]]
extraTest/testFox/3294.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.77809674]]
extraTest/testFox/3256.jpg_

In [11]:
directory = 'extraTest/testPossum'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testPossum/1980.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.15747124]]
extraTest/testPossum/1993.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.13900962]]
extraTest/testPossum/1978.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.38496742]]
extraTest/testPossum/1937.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.0092765]]
extraTest/testPossum/1995.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.39827627]]
extraTest/testPossum/1986.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.00628235]]
extraTest/testPossum/1974.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.01812453]]
extraTest/testPossum/1967.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.0214281]]
extraTest/

In [12]:
directory = 'extraTest/testRabbit'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testRabbit/735.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.08538529]]
extraTest/testRabbit/769.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.34582832]]
extraTest/testRabbit/726.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.13383855]]
extraTest/testRabbit/720.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.08160203]]
extraTest/testRabbit/733.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.3919619]]
extraTest/testRabbit/765.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.43296713]]
extraTest/testRabbit/739.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.17923701]]
extraTest/testRabbit/776.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.46208543]]
extraTest/testRab

In [13]:
directory = 'extraTest/testSquirrel'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testSquirrel/85.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 25ms/step
[[0.65878093]]
extraTest/testSquirrel/83.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.32964438]]
extraTest/testSquirrel/118.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.68556166]]
extraTest/testSquirrel/89.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.73328114]]
extraTest/testSquirrel/125.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.29272687]]
extraTest/testSquirrel/97.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.1870284]]
extraTest/testSquirrel/122.jpg___crop03_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.6637602]]
extraTest/testSquirrel/101.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.6160447]]
extraTe

In [14]:
directory = 'extraTest/testDog'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testDog/5654.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 14ms/step
[[0.91322184]]
extraTest/testDog/5645.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.09741744]]
extraTest/testDog/5657.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 14ms/step
[[0.01988441]]
extraTest/testDog/5668.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 33ms/step
[[0.4833359]]
extraTest/testDog/5670.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 16ms/step
[[0.4153719]]
extraTest/testDog/5690.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 16ms/step
[[0.24862231]]
extraTest/testDog/5667.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.38027102]]
extraTest/testDog/5648.

In [15]:
directory = 'extraTest/testCat'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        newPic.save(f[:-4] + "FlippedImage" + f[-4:])
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testCat/7102.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg


1/1 [==============================] - 0s 15ms/step
[[0.04209998]]
extraTest/testCat/7076.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.7355787]]
extraTest/testCat/7051.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 18ms/step
[[0.14323694]]
extraTest/testCat/7089.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 15ms/step
[[0.28212517]]
extraTest/testCat/7093.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 14ms/step
[[0.87103915]]
extraTest/testCat/7043.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.15600438]]
extraTest/testCat/7051.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.10748865]]
extraTest/testCat/7044.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.60

In [16]:
directory = 'extraTest/testCoyote'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        newPic.save(f[:-4] + "FlippedImage" + f[-4:])
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testCoyote/58ac0d93-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.9116945]]
extraTest/testCoyote/58b2edb2-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 15ms/step
[[0.97066635]]
extraTest/testCoyote/5a049367-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 15ms/step
[[0.7124092]]
extraTest/testCoyote/5a263855-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 15ms/step
[[0.8702231]]
extraTest/testCoyote/5a096aa5-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 14ms/step
[[0.18747504]]
extraTest/testCoyote/5a27db76-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 14ms/step
[[0.976763

## Code for data manipulation to creaet more data

In [17]:
# import os
# from PIL import Image
# directory = 'extraTraining/cropped_dog' #Just change directory name for different animal
 
# # iterate over files in
# # that directory
# results = [0,0,0,0]
# for filename in os.listdir(directory):
#     f = os.path.join(directory, filename)
#     # checking if it is a file
#     if os.path.isfile(f):
#         print(f)
#         testPicture = load_img(f, target_size=(224,224))
#         newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
#         newPic.save(f[:-4] + "FlippedImage" + f[-4:])